* Utils

In [47]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

* Problem Data

In [48]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'NFLX', 'ABNB']
data = yf.download(tickers, start="2020-01-01", end="2024-01-01")['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()
expected_returns = returns.mean()
cov_matrix = returns.cov()

[*********************100%%**********************]  5 of 5 completed


In [49]:
expected_returns = np.array([0.09, 0.12, 0.15, 0.06])
cov_matrix = np.array([
    [0, -0.5, 0.35, 0],
    [0, 0, 0.25, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
])
var = np.array([0.2, 0.3, 0.35, 0.15])

In [50]:
print("Expected return: ")
pd.DataFrame(expected_returns).head()

Expected return: 


,0
0,0.09
1,0.12
2,0.15
3,0.06


In [51]:
print("\nCovariance: ")
pd.DataFrame(cov_matrix).head()


Covariance: 


,0,1,2,3
0,0.0,-0.5,0.35,0.0
1,0.0,0.0,0.25,0.0
2,0.0,0.0,0.00,0.0
3,0.0,0.0,0.00,0.0


* Setting

In [63]:
desired_variance = 0.1
num_assets = len(expected_returns)

# Utils
def portfolio_return(weights):
    return np.dot(weights, expected_returns)

def portfolio_variance(weights, means, std_devs, correlations):
    num_assets = len(weights)
    variance = 0

    # Compute the squared product of weights and means for each asset
    weighted_mean_squared = np.square(weights) * np.square(means)
    variance += np.sum(weighted_mean_squared)
    
    # Compute the covariance terms
    for i in range(num_assets):
        for j in range(i + 1, num_assets):
            covariance_term = 2 * weights[i] * weights[j] * std_devs[i] * std_devs[j] * correlations[i, j]
            variance += covariance_term
    
    return variance
#def portfolio_variance(weights):
#    return np.dot(weights.T, np.dot(cov_matrix, weights))

def negative_portfolio_return(weights):
    return -portfolio_return(weights)

* Constraint

In [64]:
# Constraints
constraints = (
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
    {'type': 'eq', 'fun': lambda weights: portfolio_variance(weights, expected_returns, var, cov_matrix) - desired_variance}
)

* Solving

In [65]:
# Bounds for weights (between 0 and 1)
bounds = tuple((0, 1) for asset in range(num_assets))

# Initial guess (equal distribution)
initial_guess = num_assets * [1. / num_assets]

# Optimization
result = minimize(
    negative_portfolio_return,
    initial_guess,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Optimal weights
optimal_weights  = result.x
optimal_return   = portfolio_return(optimal_weights)
optimal_variance = portfolio_variance(optimal_weights, expected_returns, var, cov_matrix)

print(np.sum(optimal_weights))
print("Optimal Weights:", optimal_weights)
print("Expected Portfolio Return:", optimal_return)
print("Portfolio Variance:", optimal_variance)


1.0000000000095766
Optimal Weights: [1.69187877e-16 2.40377709e-01 7.59622291e-01 4.24545738e-18]
Expected Portfolio Return: 0.14278866872169155
Portfolio Variance: 0.0234014423073965
